# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

So models refers to the language models underpinning a 
lot of it. Prompts refers to 
the style of creating inputs to pass into the models. 
And then parsers is on the opposite end. 
It involves taking the output of these models 
and parsing it into a more structured format 
so that you can do things downstream with it. 

When you build an application using LLM, there'll 
often be reusable models. We repeatedly prompt a model, parses 
outputs, and so LangChain gives an easy set of abstractions to 
do this type of operation.

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


With this helper function, you can 
say get completion on what is 1 plus 1, and this 
will call ChatGPT, or technically the model, GPT 3.5 Turbo, to 
give you an answer back

In [ ]:
get_completion("What is 1+1?")

*OUTPUT*
```python
'1+1 equals 2.'
```

### English pirate language ask this LLM to translate the text to American English in a calm and respectful tone

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """American English \
in a calm and respectful tone
"""

Little bit of prompting before:

I'm going to specify the prompt using an f-string with 
the instructions, translate the text that is 
delimited by triple backticks into style that is style, 
and then plug in these two styles. 
And so, this generates a prompt that says translate the 
text and so on.

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

*OUTPUT*

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone.
```
text:```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```
```

In [ ]:
response = get_completion(prompt)

In [ ]:
response

*OUTPUT*

'I am quite frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add to my frustration, the warranty does not cover the cost of cleaning up my kitchen. I kindly request your assistance at this moment, my friend.'

So, if you have different customers writing 
reviews in different languages, not just English pirate, but French, 
German, Japanese, and so on, you can imagine having to generate 
a whole sequence of prompts to generate such translations. Let's 
look at how we can do this in a more convenient, way 
using LangChain

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

 I'm going to import chat OpenAI. This is 
LangChain's abstraction for the chatGPT API endpoint. 
 
And so, if I then set chat equals chat OpenAI 
and look at what chat is, it creates this object as follows 
that uses the chatGPT model, which is also called GPT 3.5 turbo. 
 

In [ ]:
from langchain.chat_models import ChatOpenAI

When I'm building applications, one thing I will often do 
is set the temperature parameter to be equal to zero. So 
the default temperature is 0.7. 
But let me actually redo that with temperature 
equals 0.0, and now the temperature is set to 0 to make this output a 
little bit less random. 

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0)
chat

*OUTPUT*
```python
ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

```

### Prompt template

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

To repeatedly reuse this template, let's import 
LangChain's chat prompt template,
and then, let me create a prompt template 
using that template string that we just wrote above.

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [ ]:
prompt_template.messages[0].prompt

*OUTPUT*
```python
PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)
```

From the prompt template, you can actually 
extract the original prompt, and it realizes that this 
prompt has two input variables, the style and the text.

In [ ]:
prompt_template.messages[0].prompt.input_variables

*OUTPUT*
```python
['style', 'text']
```

This is a style that 
I want the customer message to be translated 
to

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

*OUTPUT*
```python
<class 'list'>
<class 'langchain.schema.HumanMessage'>
```

So if you want to look at the types, 
the customer message is actually a list, 
and if you look at the first element of the list, 
this is more or less that prompt that you would 
expect this to be creating. 

In [ ]:
print(customer_messages[0])

*OUTPUT*
```python
content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False
```

Let's pass this prompt to the LLM, so I'm going 
to call chat, which we had set earlier, 
as a reference to the OpenAI chatGPT endpoint, 
and, if we print out the customer responses content, 
then, it gives you back this text translated from English 
pirate to polite American English.

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)

*OUTPUT*

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!

Now they're polite message, but 
let's say this is what a customer service agent wants. 
We are going to specify 
that the service message is going to be 
translated to this pirate style. So we want it to 
be in a polite tone that speaks in 
English pirate. 
And because we previously created that prompt template, 
the cool thing is, we can now reuse that prompt template and 
specify that the output style we want is 
this service style pirate 
and the text is this service reply.

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

*OUTPUT*

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. 
```
text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!```
```

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

*OUTPUT*

Ahoy there, matey! Me apologies, but the warranty be not coverin' the costs o' cleanin' yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to secure the lid afore ye started blendin'. Aye, tough luck, me heartie! Farewell and fair winds to ye!

![Templates](immagini/04_templates_1.png)
![Templates](immagini/04_templates_2.png)

So, you might be wondering, why are we using 
prompt templates instead of, you know, just an f-string? The 
answer is that as you build sophisticated applications, 
prompts can be quite long and detailed. 
 
And so, prompt templates are a useful abstraction to help you 
reuse good prompts when you can. 
This is an example of a relatively long 
prompt to grade a student's submission 
for an online learning application. 
And a prompt like this can be quite long, in which you can ask the 
LLM to first solve the problem, and then have the 
output in a certain format, and the output in a certain format. 
 
And wrapping this in a LangChain prompt makes 
it easier to reuse a prompt like this. 
Also, you see later that LangChain provides 
prompts for some common operations, such as summarization, or 
question answering, or connecting to SQL databases, 
or connecting to different APIs. And so by using some of LangChain's 
built-in prompts, you can quickly get an application working without 
needing to, 
engineer your own prompts.



One other aspect of LangChain's prompt libraries 
is that it also supports __output parsing__, which we'll get to in a minute. 
But when you're building a complex application using an LLM, 
you often instruct the LLM 
to generate its output in a certain format, 
such as using specific keywords. This example on 
the left illustrates using an LLM to carry out something called chain 
of thought reasoning 
using a framework called the __ReAct__ framework. But don't 
worry about the technical details, but the keys of that is that 
- the __THOUGHT__ is what the LLM 
is thinking, 
because by giving an LLM space to think, it can often get 
to more accurate conclusions. 
- Then __ACTION__ as a keyword to carry the specific action, 
and 
- then __OBSERVATION__ to show what it learned from that action, 
and so on. 
- And if you have a prompt that instructs the LLM 
to use these specific keywords, thought, action, and observation, 
then this prompt can be coupled with a parser 
to extract out the text that has been 
tagged with these specific keywords.

And so that together 
gives a very __nice abstraction to specify the 
input to an LLM__, 
and then also have a parser correctly interpret 
the output that the LLM gives.

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [ ]:
# Python dictionary
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

*OUTPUT*
```
{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}
```

How you can have 
an LLM output JSON, 
and use LangChain to parse that output. 
And the running example that I'll use will be 
to extract 
information from a product review, 
and format that output in a JSON format.

The 
review template asks the LLM 
to take as input a customer review 
and extract 
these three fields and then format the output as JSON 
with the following keys.

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

Wrap this in 
LangChain.

Let's import the chat prompt template.  and then have the prompt templates 
created from 
the review template 
up on top.

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

*OUTPUT*

```
input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]
```

Create the OpenAI endpoint, call that endpoint, and then let's 
print out the response. 

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

*OUTPUT*
```json
{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```

__! NOTE__  if we 
check the type 
of the response, this is actually a string. 
So it looks like JSON and looks like it has key-value 
pairs, but it's actually not a dictionary. This is 
just 
one long string. 

In [ ]:
type(response.content)

*OUTPUT*

str

In [ ]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

```
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[34], line 4
      1 # You will get an error by running this line of code 
      2 # because'gift' is not a dictionary
      3 # 'gift' is a string
----> 4 response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'
```

So what I'd really like to do is go to the 
response content and get the value from the 
gift key which should be true, but I run this, this 
should generate an error 
because, well, this is actually a string. 
This is not a Python dictionary.

### Parse the LLM output string into a Python dictionary

In [ ]:
# import response schema and structured output parser from LangChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

I've specified the schema for these LangChain can actually 
give you the prompt 
itself by 
having the output parser tell you what instructions it wants you 
to send to the LLM. So if I were to print 
format instructions. 
She has a pretty precise set of instructions for the LLM that 
will cause it to generate an output that the output parser 
can process. 

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()


In [ ]:
print(format_instructions)

*OUTPUT*
```
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```
```

The review template includes the 
format instructions that LangChain 
generated.

We can create a prompt 
from the review template too, 
and then create the 
messages that will pass 
to the OpenAI endpoint.

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

*OUTPUT*

```
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```
```

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

*OUTPUT*
```
```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```
```

Parse this into an output dictionary

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

*OUTPUT*
```python
{'gift': False,
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}
```

In [ ]:
type(output_dict)

*OUTPUT*

dict

In [ ]:
output_dict.get('gift')

*OUTPUT*

True

In [ ]:
output_dict.get('delivery_days')

*OUTPUT*

'2'

This is a nifty way to take your LLM output and parse it 
into a Python dictionary, 
to make the output easier to use in downstream processing. 

With these tools, hopefully you'll be able to 
reuse your own prompt templates easily, share prompt templates 
with others that you're collaborating with, even 
use LangChain's built-in prompt templates, which as you 
just saw, can often be coupled with an output 
parser, 
so that the input prompt to output 
in a specific format and then the parser, 
parses that output to store the data in a Python dictionary or 
some other data structure that makes it easy for 
downstream processing.